In [51]:
from itertools import product
import pandas as pd
import numpy as np
import re
import cv2
from sudoku import solve
import recognize
import os
from IPython import display

In [52]:

# ディレクトリ内のファイルを取得する
dir_path = "./data/level1/"
files = os.listdir(dir_path)
# jpgファイルのみを取得する
jpg_files = [os.path.splitext(f)[0] for f in files if f.endswith('.jpg')]

# ファイル名を表示する
for f in jpg_files:
    print(f)

file="sample"
# how solve works
image_path = f"./{dir_path}{file}.jpg"
image = cv2.imread(image_path, cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# solve(image)
problem=np.array(recognize.recognize(image))
problem

sample
sudoku_026
sudoku_027
sudoku_028
sudoku_029
sudoku_030


array([[0, 3, 5, 0, 9, 0, 0, 4, 8],
       [0, 0, 9, 0, 0, 8, 0, 0, 3],
       [0, 4, 0, 6, 0, 5, 0, 0, 1],
       [0, 0, 0, 0, 7, 4, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 6, 0],
       [0, 0, 0, 1, 5, 0, 0, 0, 0],
       [8, 0, 0, 9, 0, 2, 0, 7, 0],
       [9, 0, 0, 5, 0, 0, 2, 0, 0],
       [6, 1, 0, 0, 4, 0, 5, 3, 0]])

In [53]:
# Knuth's Algorithm X
def dfs(df: pd.DataFrame):
    # 使い勝手のためにNumPy配列に変換
    m = df.to_numpy()
    if m.size == 0:
        return []

    # 1の数が最小の行を探す
    sums = np.sum(m, axis=0)
    sc = np.argmin(sums)
    if sums[sc] == 0:
        return None

    sel_rows = np.where(m[:, sc])[0]
    for sr in sel_rows:
        mask_cols = np.where(m[sr, :])[0]
        mask_rows = [np.where(m[:, c])[0] for c in mask_cols]
        mask_rows = np.unique(np.concatenate(mask_rows))

        new_df = df.drop(index=df.index[mask_rows]).drop(columns=df.columns[mask_cols])
        ret = dfs(new_df)
        if ret is not None:
            return ret + [df.index[sr]]

    return None

In [54]:
index = [
    "R{:d}C{:d}#{:d}".format(i, j, n)
    for i in range(1, 10)
    for j in range(1, 10)
    for n in range(1, 10)
]
cond1 = ["R{:d}C{:d}".format(i, j) for i in range(1, 10) for j in range(1, 10)]
cond2 = ["R{:d}#{:d}".format(i, n) for i in range(1, 10) for n in range(1, 10)]
cond3 = ["C{:d}#{:d}".format(j, n) for j in range(1, 10) for n in range(1, 10)]
cond4 = ["B{:d}#{:d}".format(b, n) for b in range(1, 10) for n in range(1, 10)]
conds = cond1 + cond2 + cond3 + cond4

df = pd.DataFrame(index=index, columns=conds, dtype="bool")
df.iloc[:] = 0
display.display(df)
df.value_counts(),df.shape

,R1C1,R1C2,R1C3,R1C4,R1C5,R1C6,R1C7,R1C8,R1C9,R2C1,...,B8#9,B9#1,B9#2,B9#3,B9#4,B9#5,B9#6,B9#7,B9#8,B9#9
R1C1#1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R9C9#5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R9C9#6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R9C9#7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R9C9#8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(R1C1  R1C2  R1C3  R1C4  R1C5  R1C6  R1C7  R1C8  R1C9  R2C1  R2C2  R2C3  R2C4  R2C5  R2C6  R2C7  R2C8  R2C9  R3C1  R3C2  R3C3  R3C4  R3C5  R3C6  R3C7  R3C8  R3C9  R4C1  R4C2  R4C3  R4C4  R4C5  R4C6  R4C7  R4C8  R4C9  R5C1  R5C2  R5C3  R5C4  R5C5  R5C6  R5C7  R5C8  R5C9  R6C1  R6C2  R6C3  R6C4  R6C5  R6C6  R6C7  R6C8  R6C9  R7C1  R7C2  R7C3  R7C4  R7C5  R7C6  R7C7  R7C8  R7C9  R8C1  R8C2  R8C3  R8C4  R8C5  R8C6  R8C7  R8C8  R8C9  R9C1  R9C2  R9C3  R9C4  R9C5  R9C6  R9C7  R9C8  R9C9  R1#1  R1#2  R1#3  R1#4  R1#5  R1#6  R1#7  R1#8  R1#9  R2#1  R2#2  R2#3  R2#4  R2#5  R2#6  R2#7  R2#8  R2#9  R3#1  R3#2  R3#3  R3#4  R3#5  R3#6  R3#7  R3#8  R3#9  R4#1  R4#2  R4#3  R4#4  R4#5  R4#6  R4#7  R4#8  R4#9  R5#1  R5#2  R5#3  R5#4  R5#5  R5#6  R5#7  R5#8  R5#9  R6#1  R6#2  R6#3  R6#4  R6#5  R6#6  R6#7  R6#8  R6#9  R7#1  R7#2  R7#3  R7#4  R7#5  R7#6  R7#7  R7#8  R7#9  R8#1  R8#2  R8#3  R8#4  R8#5  R8#6  R8#7  R8#8  R8#9  R9#1  R9#2  R9#3  R9#4  R9#5  R9#6  R9#7  R9#8  R9#9  C1#1  C1#2  C1#3  C1#4  C1#

In [55]:
#　すべて空マスの場合の条件
for i in range(9):
    for j in range(9):
        for n in range(1, 10):
            r = i + 1
            c = j + 1
            b = (i // 3) * 3 + (j // 3) + 1
            op = "R{:d}C{:d}#{:d}".format(r, c, n)
            cd1 = "R{:d}C{:d}".format(r, c)
            cd2 = "R{:d}#{:d}".format(r, n)
            cd3 = "C{:d}#{:d}".format(c, n)
            cd4 = "B{:d}#{:d}".format(b, n)
            df.loc[op, [cd1, cd2, cd3, cd4]] = 1

display.display(df)

,R1C1,R1C2,R1C3,R1C4,R1C5,R1C6,R1C7,R1C8,R1C9,R2C1,...,B8#9,B9#1,B9#2,B9#3,B9#4,B9#5,B9#6,B9#7,B9#8,B9#9
R1C1#1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R9C9#5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
R9C9#6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
R9C9#7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
R9C9#8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [56]:

for i in range(9):
    for j in range(9):
        if problem[i, j] != 0:
            r = i + 1
            c = j + 1
            b = (i // 3) * 3 + (j // 3) + 1
            n = problem[i, j]
            # print(n)
            del_op1 = ["R{:d}C{:d}#{:d}".format(r, c, n_) for n_ in range(1, 10) if n != n_]
            del_op2 = ["R{:d}C{:d}#{:d}".format(r_, c, n) for r_ in range(1, 10) if r != r_]
            del_op3 = ["R{:d}C{:d}#{:d}".format(r, c_, n) for c_ in range(1, 10) if c != c_]
            del_op4 = [
                "R{:d}C{:d}#{:d}".format(r_, c_, n)
                for r_ in range(i // 3 * 3 + 1, i // 3 * 3 + 4)
                for c_ in range(j // 3 * 3 + 1, j // 3 * 3 + 4)
                if r != r_ or c != c_
            ]

            del_op = list(set(del_op1 + del_op2 + del_op3 + del_op4))
            df.drop(index=[id for id in del_op if id in df.index], inplace=True)

display.display(df)
# df.value_counts()

,R1C1,R1C2,R1C3,R1C4,R1C5,R1C6,R1C7,R1C8,R1C9,R2C1,...,B8#9,B9#1,B9#2,B9#3,B9#4,B9#5,B9#6,B9#7,B9#8,B9#9
R1C1#1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C1#7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C2#3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1C3#5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R9C5#4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R9C6#7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R9C7#5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
R9C8#3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [57]:
# Knuth's Algorithm X
def dfs(df: pd.DataFrame):
    # 使い勝手のためにNumPy配列に変換
    m = df.to_numpy()
    if m.size == 0:
        return []

    # 1の数が最小の行を探す
    sums = np.sum(m, axis=0)
    selected_col = np.argmin(sums)
    if sums[selected_col] == 0:
        return None

    sel_rows = np.where(m[:, selected_col])[0]
    for sr in sel_rows:
        mask_cols = np.where(m[sr, :])[0]
        mask_rows = [np.where(m[:, c])[0] for c in mask_cols]
        mask_rows = np.unique(np.concatenate(mask_rows))

        new_df = df.drop(index=df.index[mask_rows]).drop(columns=df.columns[mask_cols])
        ret = dfs(new_df)
        if ret is not None:
            return ret + [df.index[sr]]

    return None

In [60]:
ans=dfs(df)
# ans
# ans

['R9C9#9',
 'R9C8#3',
 'R9C7#5',
 'R9C6#7',
 'R9C5#4',
 'R9C4#8',
 'R9C3#2',
 'R9C2#1',
 'R9C1#6',
 'R8C9#6',
 'R8C8#8',
 'R8C7#2',
 'R8C5#1',
 'R7C9#4',
 'R7C5#6',
 'R6C9#2',
 'R6C8#9',
 'R6C7#3',
 'R6C2#8',
 'R4C9#5',
 'R4C8#1',
 'R4C2#9',
 'R4C7#8',
 'R7C7#1',
 'R6C1#4',
 'R5C9#7',
 'R4C1#3',
 'R5C1#5',
 'R5C7#4',
 'R4C3#6',
 'R5C3#1',
 'R6C3#7',
 'R7C3#3',
 'R8C3#4',
 'R8C6#3',
 'R8C4#5',
 'R6C6#6',
 'R5C6#9',
 'R2C5#2',
 'R2C4#4',
 'R2C1#1',
 'R1C1#2',
 'R1C6#1',
 'R1C4#7',
 'R4C4#2',
 'R5C4#3',
 'R5C5#8',
 'R3C5#3',
 'R3C3#8',
 'R3C1#7',
 'R2C8#5',
 'R3C8#2',
 'R3C7#9',
 'R1C7#6',
 'R2C7#7',
 'R2C2#6',
 'R8C2#7',
 'R8C1#9',
 'R7C8#7',
 'R7C6#2',
 'R7C4#9',
 'R7C2#5',
 'R7C1#8',
 'R6C5#5',
 'R6C4#1',
 'R5C8#6',
 'R5C2#2',
 'R4C6#4',
 'R4C5#7',
 'R3C9#1',
 'R3C6#5',
 'R3C4#6',
 'R3C2#4',
 'R2C9#3',
 'R2C6#8',
 'R2C3#9',
 'R1C9#8',
 'R1C8#4',
 'R1C5#9',
 'R1C3#5',
 'R1C2#3']

In [69]:
answ=np.ones((9,9))
for a in ans:
    # print(a)
    _,r,c,n= re.split("[RC#]", a)
    answ[int(r) - 1, int(c) - 1] = int(n)
answ

array([[2., 3., 5., 7., 9., 1., 6., 4., 8.],
       [1., 6., 9., 4., 2., 8., 7., 5., 3.],
       [7., 4., 8., 6., 3., 5., 9., 2., 1.],
       [3., 9., 6., 2., 7., 4., 8., 1., 5.],
       [5., 2., 1., 3., 8., 9., 4., 6., 7.],
       [4., 8., 7., 1., 5., 6., 3., 9., 2.],
       [8., 5., 3., 9., 6., 2., 1., 7., 4.],
       [9., 7., 4., 5., 1., 3., 2., 8., 6.],
       [6., 1., 2., 8., 4., 7., 5., 3., 9.]])